In [4]:
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
import scipy.sparse as sp
from pathlib import Path

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [5]:
discard_dataset_path = "data\\discardDataSets"
dataset_folder = Path(discard_dataset_path) / "2019" #path to file folder
print(dataset_folder)

data\discardDataSets\2019


In [27]:
file_train = list(dataset_folder.iterdir())[0]       #path to single file
file_test = list(dataset_folder.iterdir())[1]

game_train = sp.load_npz(file_train)     #loading file
game_test = sp.load_npz(file_test)

matrix_train = game_test.toarray()  #converting to array
matrix_test = game_test.toarray()

In [45]:
#function to convert y into one-hot format for model training

def one_hot(array):
    out = np.zeros((array.size, 34))
    out[np.arange(array.size), array] = 1
    return out

In [47]:
X_train = matrix_train[:, :510]    # board state as a big ass array 
y_train = one_hot( matrix_train[:, -1] )     # discarded tile (label)

X_test = matrix_test[:, :510]
y_test = one_hot( matrix_test[:, -1] ) 

In [58]:
model = Sequential([
    Dense(64, input_shape = (510, ), activation="relu"),
    Dense(64, activation="relu"),
    Dense(34, activation="softmax"),
])

d:\GitHub\MahjongBot2.0\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [59]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',  # or 'sparse_categorical_crossentropy' if your labels are not one-hot encoded
              metrics=['accuracy'])

In [60]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0372 - loss: 24.6556 - val_accuracy: 0.0277 - val_loss: 9.2474
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0116 - loss: 8.0718 - val_accuracy: 0.0490 - val_loss: 4.7679
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0546 - loss: 4.5574 - val_accuracy: 0.0576 - val_loss: 4.1666
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0444 - loss: 4.1460 - val_accuracy: 0.0405 - val_loss: 3.6878
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0524 - loss: 3.6524 - val_accuracy: 0.0554 - val_loss: 3.5854
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0372 - loss: 3.6668 - val_accuracy: 0.0426 - val_loss: 3.4630
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0409 - loss: 3.5368 - val_accuracy: 0.0490 - val_loss: 3.4854
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0531 - loss: 3.5132 - val_accuracy: 0.0725 - val_los

In [63]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.1239 - loss: 3.2055
Test Loss: 3.240421772003174
Test Accuracy: 0.10660980641841888


In [65]:
test_loss, test_accuracy = model.evaluate(X_train, y_train)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.1239 - loss: 3.2055
Test Loss: 3.240421772003174
Test Accuracy: 0.10660980641841888
